# Classification de plantes

## Imports

### Library

In [1]:
# Imports de base
import pandas as pd

### Supprimer les warnings futurs

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Chargement des données

In [5]:
file = "../data/raw_data/metadata.csv"
df = pd.read_csv(file,sep=",")
df.head()

,observationid,mediaid,vote,content,classid,family,genus,species,author,date,location,latitude,longitude,yearinclef,observationid2014,imageid2014,learntag
0,15010,100019,3.0,Flower,8527,Asteraceae,Achillea,Achillea millefolium L.,mathieu menand,2009-4-24,Aucamville,NaN,NaN,PlantCLEF2014,4688.0,52287.0,Train
1,10732,100056,4.0,Flower,4516,Ranunculaceae,Ficaria,Ficaria verna Huds.,alain06,2014-9-7,Biot,43.62495,7.10440,PlantCLEF2015,NaN,NaN,Train
2,24005,100087,4.0,Flower,588,Asteraceae,Centaurea,Centaurea jacea L.,thierry pernot,1800-1-1,NaN,NaN,NaN,PlantCLEF2014,2559.0,32864.0,Train
3,19892,100182,4.0,Flower,9930,Orchidaceae,Epipactis,Epipactis helleborine (L.) Crantz,thierry pernot,1800-1-1,NaN,NaN,NaN,PlantCLEF2014,11733.0,40782.0,Train
4,36220,10019,4.0,Flower,6487,Orchidaceae,Ophrys,Ophrys apifera Huds.,jean-claude echardour,2010-6-26,Saffré,47.49689,-1.59442,PlantCLEF2014,13210.0,8080.0,Train


### Combien de données (photos) on a pour chaque espèce

In [6]:
df.groupby('species')['mediaid'].count()

species
Achillea millefolium L.                                     62
Aconitum napellus L.                                        61
Anacamptis morio (L.) R.M.Bateman, Pridgeon & M.W.Chase     49
Anacamptis pyramidalis (L.) Rich.                           85
Anemone alpina L.                                           52
Anemone hepatica L.                                         52
Anemone nemorosa L.                                         55
Aquilegia vulgaris L.                                       75
Astrantia major L.                                          56
Bellis perennis L.                                          97
Centaurea aspera L.                                         65
Centaurea jacea L.                                          66
Chelidonium majus L.                                        60
Cichorium intybus L.                                       109
Cirsium vulgare (Savi) Ten.                                 58
Cistus albidus L.                              

### Nombre total de données

In [7]:
tailleTotale = df.mediaid.unique().size
print(tailleTotale)

3474


## Plan des étapes à faire
    Faire la somme cumulée des plantes : si y'a un gros pic => on a une espècee qui représente une grosse partie des données
    
    Est ce que les données sont bien balancées? 
    
    Métrique: comment comparer les différents modèles? 
    Si données pas trop balancées: accuracy pas trop mal pour mesurer!
    Moyenner par espèce: taux moyen pour chaque espèce
    Mesurer le taux d'erreur : données non balancées (issues de l'app plantNet: attiré plus vers certaines plantes plutot que d'autres) 
    stratifySplit (split, trainset split) 
    
    1) Prendre une baseline basique : 
        - Prendre un classifieur le plus naif possible et calculer son accuracy et on aura un accuracy de base 
        - permet de donner un sens aux chiffres 
        - Si moins bien que le truc très naif c'est qu'il y a un soucis
    2) Mettre les images en entrée et de regarder ce qui sort d'un classifier linéaire. 
        a) Redimensionner : vectoriser 
        b) Balancer au classifier: tableau 3D 
    3) https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_hog.html#sphx-glr-auto-examples-features-detection-plot-hog-py
    Celui ci retire les couleurs: importants pour les fleurs
    Descripteur global sur image : si rotation alors problème , baisse de luminosité etc..
    
    Combiner les descripteurs : hog combiné à un descripteur de couleurs
    
    4) Bag-of-Visual-Words (BoVW) : Prend les descripteurs et en fait une représentation (marche plutot bien)
    
    5) Réseau neuronal : passer en entrée les images et lui va apprendre les features
    Pytorch: CNN (exemples tutos) 
    MLP (Multi Layer perceptron : bcp plus rapide que le convutionnel)
    Transfer learning : prendre un autre modele déja entrainé et le réutiliser : utile qd on a peu de données
    Meilleur pour réseau neuronnaux de partir d'un modèle pré-enrtainé

## Classification naïve: prédire toujours même classe (BaseLine)

C'est le classifieur le plus bas : si on a plu bas c'est qu'il y a un soucis qqs part.

Papaver rhoeas L.                                          119 (à prédire)

Calculer accuracy , f-measure, rappel

Accuracy = nbre d'éléments corrects / nombre total
Précision = nbre d'éléments corrects pour la classe / nbre prédit
Rappel = nbre d'éléments correctement attribués à la classe / nbre de documents appartenant à la classe i (ici 119) 

In [8]:
# Définition de la prédiction qui prédit toujours la même classe (avec la classe qui a le plus de photo)
def naivePredict(row):
    return 30269

In [13]:
# Application de la prédiction naïve sur toutes les lignes du dataframe
def predictionNaive(df):
    count = 0
    for index, row in df.iterrows():
        res = naivePredict(row)
        if (res == row["classid"]):
            count = count + 1
    #print("count : ", count)
    acc = count / tailleTotale
    prec = count / tailleTotale
    rappel = count / 119
    print("Accuracy : ", acc)
    print("Précision : ", prec)
    print("Rappel : ", rappel)
    

In [14]:
# Test prediction naive : 
predictionNaive(df)

Accuracy :  0.03425446171560161
Précision :  0.03425446171560161
Rappel :  1.0


## Classification sans features (SVM, LR)

Réduire la taille de l'image car taille d'image trop grande (ne rentrera pas en mémoire) => descripteur de taille + petite